In [8]:
import pandas as pd
from common.utils import Properties, Config

In [2]:
# raw_df = pd.read_csv('../housing_raw_data/new-york-city_listings.csv')
# pd.set_option('display.max_columns', len(raw_df.columns))
# pd.set_option('display.max_rows', 100)
# raw_df.head(3)

F:\Developer\anaconda\envs\airbnb\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75356,-73.98559,Entire home/apt,150,30,48,2019-11-04,0.33,3,334,0,NaN
1,3831,"Whole flr w/private bdrm, bath & kitchen(pls r...",4869,LisaRoxanne,Brooklyn,Bedford-Stuyvesant,40.68494,-73.95765,Entire home/apt,73,1,409,2021-10-22,4.62,1,214,26,NaN
2,5121,BlissArtsSpace!,7356,Garon,Brooklyn,Bedford-Stuyvesant,40.68535,-73.95512,Private room,60,30,50,2019-12-02,0.33,2,365,0,NaN


In [9]:
def count_one_city(city: str) -> dict:
    raw_df = pd.read_csv('../housing_raw_data/{}_listings.csv'.format(city))
    prices = {}
    for index, row in raw_df.iterrows():
        room_type = row['room_type']
        price = row['price'] * 1.0
        if room_type:
            if room_type in prices:
                count = prices[room_type][1]
                prices[room_type][0] = (prices[room_type][0] * count + price) / (count + 1)
                prices[room_type][1] += 1
            else:
                weights[room_type] = [price, 1]
    print(weights)
    return weights

In [10]:
def union_weights(prices_major: dict, prices_add: dict):
    for key, value in prices_add.items():
        if key in prices_major:
            count = prices_major[key][1]
            count_add = value[1]
            price_major = prices_major[key][0]
            prices_major[key][0] = (price_major * count + value[0] * count_add) / (count + count_add)
            prices_major[key][1]+=count_add
        else:
            prices_major[key] = [value[0], value[1]]

In [12]:
config = Config.get_instance()
valid_cities = config['housing_valid_cities'].split('/')
prices = {}
for city in valid_cities:
    local_prices = count_one_city(city)
    union_weights(prices, local_prices)
print("overall prices: {}".format(prices))

C:\Users\51370\AppData\Local\Temp/ipykernel_17068/3724728268.py:5: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  local_prices = count_one_city(city)


{'Entire home/apt': [210.9671181782492, 20376], 'Private room': [96.80268874016684, 17034], 'Hotel room': [237.0717703349283, 209], 'Shared room': [150.48409893992937, 566]}
{'Entire home/apt': [323.75637601240567, 23212], 'Private room': [126.86790110685814, 9667], 'Hotel room': [498.8899082568808, 109], 'Shared room': [62.91108247422685, 776]}
{'Private room': [78.40527283874933, 1631], 'Entire home/apt': [199.1400377913084, 4763], 'Shared room': [47.57303370786517, 89], 'Hotel room': [313.57377049180326, 61]}
{'Entire home/apt': [330.5003787878791, 2640], 'Private room': [139.36191425722825, 1003], 'Shared room': [404.125, 8], 'Hotel room': [159.69230769230768, 13]}
overall prices: {'Entire home/apt': [267.39471671471375, 50991], 'Private room': [107.14259417078576, 29335], 'Hotel room': [319.2117346938776, 392], 'Shared room': [98.3043780403058, 1439]}


In [18]:
ratios = {}
std_price = prices['Entire home/apt'][0]
# for _, price in prices.items():
#     total_price+=price[0]

for room_type, price in prices.items():
    ratios[room_type] = std_price/price[0]

print("ratios: {}".format(ratios))


ratios: {'Entire home/apt': 1.0, 'Private room': 2.4956901481075344, 'Hotel room': 0.8376719514122621, 'Shared room': 2.720069258818556}
